In [1]:
include("data/10_ulysses_3.tsp")
n,L,B,K,W_v,w_v,W,coordinates

(10, 2, 87, 3, [1.47239, 0.82384, 0.147042, 1.11452, 1.35826, 1.95929, 1.98969, 1.54618, 2.14362, 1.6074], [4, 14, 3, 2, 20, 5, 14, 10, 10, 18], 9, [38.24 20.42; 39.57 26.15; … ; 41.23 9.1; 41.17 13.05])

In [2]:
using JuMP
using CPLEX

In [3]:
function distance(n::Int64,coordinates::Matrix{Float64})
    l=Matrix{Float64}(zeros(n,n))
    for i in 1:n
        for j in 1:n
            l[i,j]=sqrt((coordinates[i,1]-coordinates[j,1])^2+(coordinates[i,2]-coordinates[j,2])^2)
        end 
    end 
    return l
end

distance (generic function with 1 method)

In [4]:
l=distance(n,coordinates)

10×10 Matrix{Float64}:
  0.0        5.88233   5.42148   3.34819  …   0.720278  11.7082    7.93107
  5.88233    0.0       1.2919    4.48743      6.06684   17.1306   13.1973
  5.42148    1.2919    0.0       4.83011      5.74943   16.2338   12.2852
  3.34819    4.48743   4.83011   0.0          2.96142   14.8749   11.2033
 10.9669    16.7559   16.3883   12.8835      10.6926     7.88265   8.08926
  8.25804   14.104    13.4684   11.007    …   8.2501     4.7976    3.71102
  7.31222   13.0906   12.3961   10.2404       7.38505    4.89655   2.75065
  0.720278   6.06684   5.74943   2.96142      0.0       11.9315    8.24224
 11.7082    17.1306   16.2338   14.8749      11.9315     0.0       3.95046
  7.93107   13.1973   12.2852   11.2033       8.24224    3.95046   0.0

In [5]:
function resolution_statique(n,L,B,K,W_v,w_v,W,coordinates)
    
    l=distance(n,coordinates)
    m=Model(optimizer_with_attributes(CPLEX.Optimizer, "CPX_PARAM_MIPDISPLAY" =>2,"CPX_PARAM_TILIM" => 1800))
    @variable(m,x[i=1:n,j=1:n],Bin)
    @variable(m,y[i=1:n,k=1:K],Bin)
    @constraint(m,[i=1:n,j=1:n,k=1:K],x[i,j]+y[i,k]-y[j,k]<=1)
    @constraint(m,[i=1:n,j=1:n,k=1:K],x[i,j]-y[i,k]+y[j,k]<=1)
    @constraint(m,[i=1:n,j=1:n,k=1:K],-x[i,j]+y[i,k]+y[j,k]<=1)
    @constraint(m,[k=1:K],sum(w_v[i]*y[i,k] for i=1:n)<=B)
    @constraint(m,[i=1:n],sum(y[i,k] for k=1:K)==1)
    @objective(m,Min,sum(l[i,j]*x[i,j] for i=1:n,j=1:n))
    return m 
end

resolution_statique (generic function with 1 method)

In [6]:
function resolution_dualite(n,L,B,K,W_v,w_v,W,coordinates)
    l=distance(n,coordinates)
    m=Model(optimizer_with_attributes(CPLEX.Optimizer, "CPX_PARAM_MIPDISPLAY" =>2,"CPX_PARAM_TILIM" => 1800))
    @variable(m,alpha>=0)
    @variable(m,beta[i=1:n,j=1:n]>=0)
    @variable(m,x[i=1:n,j=1:n],Bin)
    @variable(m,y[i=1:n,k=1:K],Bin)
    @variable(m,gamma[k=1:K]>=0)
    @variable(m,phi[i=1:n,k=1:K]>=0)



    @constraint(m,[i=1:n,j=1:n,k=1:K],x[i,j]+y[i,k]-y[j,k]<=1)
    @constraint(m,[i=1:n,j=1:n,k=1:K],x[i,j]-y[i,k]+y[j,k]<=1)
    @constraint(m,[i=1:n,j=1:n,k=1:K],-x[i,j]+y[i,k]+y[j,k]<=1)
    @constraint(m,[i=1:n,j=1:n],alpha+beta[i,j]>=(lh[i]+lh[j])*x[i,j])
    @constraint(m,[k=1:K],sum(w_v[i]*y[i,k]+W_v[i]*phi[i,k] for i=1:n)+W*gamma[k]<=B)
    @constraint(m,[i=1:n,k=1:K],gamma[k]+phi[i,k]>=w_v[i]*y[i,k])
    @constraint(m,[i=1:n],sum(y[i,k] for k=1:K)==1)

    @objective(m,Min,sum(l[i,j]*x[i,j]+3*beta[i,j] for i=1:n,j=1:n)+L*alpha)
    return m
end

resolution_dualite (generic function with 1 method)

In [8]:
include("data/10_ulysses_9.tsp")
ms=resolution_statique(n,L,B,K,W_v,w_v,W,coordinates)
optimize!(ms)

In [ ]:
md=resolution_dualite(n,L,B,K,W_v,w_v,W,coordinates)
optimize!(md)

In [ ]:
data=["data/10_ulysses_3.tsp","data/10_ulysses_6.tsp","data/10_ulysses_9.tsp","data/14_burma_3.tsp","data/14_burma_6.tsp","data/14_burma_9.tsp","data/22_ulysses_3.tsp","data/22_ulysses_6.tsp","data/22_ulysses_9.tsp","data/26_eil_3.tsp","data/26_eil_6.tsp","data/26_eil_9.tsp","data/30_eil_3.tsp","data/30_eil_6.tsp","data/30_eil_9.tsp"]

In [ ]:
for path in data 
    open("results_statique.txt","a") do file 
        include(path)
        ms=resolution_statique(n,L,B,K,W_v,w_v,W,coordinates)
        optimize!(ms)
        println(file,string(n)*"&"*string(K)*"&"*string(objective_value(ms))*"&"*string(solve_time(ms)))
    end
end

In [ ]:
for path in data 
    open("results_dualité.txt","a") do file 
        include(path)
        ms=resolution_dualite(n,L,B,K,W_v,w_v,W,coordinates)
        optimize!(ms)
        println(file,string(n)*"&"*string(K)*"&"*string(objective_value(ms))*"&"*string(solve_time(ms)))
    end
end